In [12]:

import openai 
import os  

from dotenv import load_dotenv


In [13]:

load_dotenv(override=True)

client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_API_BASE")
)

print("API Key:", client.api_key[:10], "...")
print("Base URL:", client.base_url)


API Key: sk-or-v1-f ...
Base URL: https://openrouter.ai/api/v1/


In [14]:
response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[
        {"role": "user", "content": "Hello AI, please introduce yourself"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)


Hello! I’m an AI language model created by OpenAI, and I’m here to assist with answering questions, providing explanations, generating ideas, or just having a conversation about almost any topic. I don’t have personal experiences or consciousness, but I’m trained on a vast amount of information to help with a wide range of tasks. Let me know how I can assist you today! 😊


In [15]:
response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[{"role": "user", "content": "What is Python in one sentence?"}],
    max_tokens=500

)

ai_text = response.choices[0].message.content 


print("\nAI's Answer:")
print(ai_text)




AI's Answer:
Python is a high-level, versatile programming language known for its simplicity, readability, and extensive libraries, making it suitable for a wide range of applications from web development to data science and artificial intelligence.


In [16]:
input_tokens = response.usage.prompt_tokens  
output_tokens = response.usage.completion_tokens  
total_tokens = response.usage.total_tokens    


print(f"  The question used: {input_tokens} tokens")
print(f"  AI's response used: {output_tokens} tokens")
print(f"  Total tokens billed: {total_tokens} tokens")




input_price_per_1k = 0.0008   
output_price_per_1k = 0.0032  


input_cost = (input_tokens / 1000) * input_price_per_1k
output_cost = (output_tokens / 1000) * output_price_per_1k
total_cost = input_cost + output_cost

print("\n Cost Breakdown for This Call:")
print(f"  Input cost:  ${input_cost:.6f} ({input_tokens} tokens)")
print(f"  Output cost: ${output_cost:.6f} ({output_tokens} tokens)")
print(f"  TOTAL COST:  ${total_cost:.6f}")



  The question used: 10 tokens
  AI's response used: 40 tokens
  Total tokens billed: 50 tokens

 Cost Breakdown for This Call:
  Input cost:  $0.000008 (10 tokens)
  Output cost: $0.000128 (40 tokens)
  TOTAL COST:  $0.000136


## LangChain

10+ lines of boilerplate. Want to switch to Google? Rewrite everything! brahhhhhh thanks to langChain

In [17]:

import os

def raw_openai_approach():
    
    import openai

    client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_API_BASE")
)

    response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[{"role": "user", "content": "Explain machine learning in one sentence"}],
    max_tokens=500

)

    if response:
        text = response.choices[0].message.content 
        print(f"Normal Client Calling : {text[:100]}...")
        return text

    return None

def langchain_approach():
    from langchain_openai import ChatOpenAI

    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",                    
        api_key=os.getenv("OPENROUTER_API_KEY"),      
        base_url=os.getenv("OPENROUTER_API_BASE"),      
        max_tokens=500, 
    )

    
    response = llm.invoke("Explain machine learning in one sentence")  

    if response:
        print(f"Langhain Response result : {response.content[:100]}...")
        return response.content

    return None

def main():
     
    raw_openai_approach()
    langchain_approach()

    
if __name__ == "__main__":
    main()

Normal Client Calling : Machine learning is a subset of artificial intelligence that enables systems to learn and improve fr...
Langhain Response result : Machine learning is the process of training algorithms to learn patterns and make predictions or dec...


In [18]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

def main():
    
    
    template = PromptTemplate(
        input_variables=["topic", "style"], 
        template="Explain {topic} in {style}" 
    )
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )

    if template and llm:
        
        test_prompt = template.format(
            topic="artificial intelligence", 
            style="exactly 5 words" 
        )

        print(f"Sending to AI: {test_prompt}")
        response = llm.invoke(test_prompt)
        print(f"\n AI Response: {response.content}")



if __name__ == "__main__":
    main()

Sending to AI: Explain artificial intelligence in exactly 5 words

 AI Response: Machines simulating human-like intelligence processes.


In [19]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

def main():
    

    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )
       

    list_parser = CommaSeparatedListOutputParser() 

  
    list_prompt = PromptTemplate(
        template=  "List 3 benefits of {technology} (comma-separated):",
        input_variables=["technology"]
    )

    list_chain = list_prompt | llm | list_parser 
   
    if list_chain:
        result = list_chain.invoke({
            "technology": "cloud computing"
        })
        print(f" Input: 'List 3 benefits of cloud computing'")
        print(f"Parsed Output: {result}")
        print(f"Type: {type(result)}")
        print(f"Access items: result[0] = '{result[0] if result else ''}'")

    
    
    json_parser = JsonOutputParser()

    json_prompt = PromptTemplate(
        template="""Analyze {technology} and respond with JSON containing:
        - benefits: array of 2 benefits
        - complexity: low/medium/high
        - use_case: one main use case

        Technology: {technology}

        {format_instructions}""",
        input_variables=["technology"],
        partial_variables={"format_instructions": json_parser.get_format_instructions()}
    )

    json_chain = json_prompt | llm | json_parser 

   
    if json_chain:
        result = json_chain.invoke({
            "technology": "machine learning"
        })

        print(f"Input: 'Analyze machine learning'")

        try:
      
            parsed = result 

            print(f"   Benefits: {parsed.get('benefits', [])}")
            print(f"   Complexity: {parsed.get('complexity', 'N/A')}")
            print(f"   Use Case: {parsed.get('use_case', 'N/A')}")
            print(f"    Type: {type(parsed)}" )
        except (json.JSONDecodeError, TypeError, AttributeError):
            print(f" Parsing failed : {result}")



if __name__ == "__main__":
    main()

 Input: 'List 3 benefits of cloud computing'
Parsed Output: ['Cost efficiency', 'scalability', 'accessibility.']
Type: <class 'list'>
Access items: result[0] = 'Cost efficiency'
Input: 'Analyze machine learning'
   Benefits: ['Automates complex decision-making processes', 'Enhances predictive accuracy based on historical data']
   Complexity: high
   Use Case: Predictive analytics in finance
    Type: <class 'dict'>


## Complete Chain - Combining Everything!


In [20]:

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser

def main():
    

    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )  
    
    analysis_prompt = PromptTemplate(
        template="Analyze {technology} and provide pros and cons in 2-3 sentences",
        input_variables=["technology"]
    )

    str_parser = StrOutputParser()

    
    analysis_chain = analysis_prompt | llm | str_parser 

    
    if analysis_chain:
        result = analysis_chain.invoke({
            "technology": "blockchain"
        })
        print(f" Input: 'Analyze blockchain'")
        print(f" Output: {result}")


   
    list_prompt = PromptTemplate(
        template="List 3 use cases for {technology} (comma-separated):",
        input_variables=["technology"]
    )

    
    list_parser = CommaSeparatedListOutputParser()

    
    list_chain = list_prompt | llm | list_parser 

    if list_chain:
        result = list_chain.invoke({
            "technology": "blockchain"
        })
        print(f" Input: 'List use cases for blockchain'")
        print(f" Output: {result}")
        print(f" Type: {type(result)} - Python list!")

    
    test_tech = "artificial intelligence"
    print(f"Technology: {test_tech}\n")

    
    if analysis_chain and list_chain:
        analysis = analysis_chain.invoke({"technology": test_tech})
        print(f" Analysis:\n   {analysis}")

        use_cases = list_chain.invoke({"technology": test_tech})
        print(f"\n Use Cases:")
        for i, use_case in enumerate(use_cases, 1):
            print(f"   {i}. {use_case}")

if __name__ == "__main__":
    main()

 Input: 'Analyze blockchain'
 Output: **Pros:** Blockchain offers enhanced security through decentralization and cryptographic hashing, ensuring data integrity and reducing fraud. It also enables transparent, tamper-proof records and eliminates intermediaries, reducing costs and increasing efficiency.  

**Cons:** Blockchain can be resource-intensive, requiring significant energy consumption for mining and maintenance. Scalability issues and slower transaction speeds compared to traditional systems can limit its widespread adoption.
 Input: 'List use cases for blockchain'
 Output: ['Here are three common use cases for blockchain technology', 'separated by commas:  ', '1. **Cryptocurrencies (e.g.', 'Bitcoin', 'Ethereum)**', '', '2. **Supply Chain Tracking & Transparency**', '', '3. **Smart Contracts & Decentralized Applications (DApps)**  ', "Let me know if you'd like more details or additional examples!"]
 Type: <class 'list'> - Python list!
Technology: artificial intelligence

 Analys

## Prompt 

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

def main():
    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )  

    
    vague_prompt = "write a policy" 

    print(f"{vague_prompt}")

    
    vague_response = llm.invoke(vague_prompt)
    print(f"\n response preview: {vague_response.content[:100]}...")
    
    specific_prompt = """Write a 200-word GDPR-compliant data privacy policy
         for European customers with 30-day retention period"""

    print(f" Specific prompt: {specific_prompt[:50]}")

    
    specific_response = llm.invoke(specific_prompt)
    print(f"\nSpecific response : {specific_response.content[:200]}...")
    

    
if __name__ == "__main__":
    main()

write a policy

 response preview: Certainly! Below is a template for a **Workplace Remote Work Policy** that you can customize based o...
 Specific prompt: Write a 200-word GDPR-compliant data privacy polic

Specific response : **GDPR-Compliant Data Privacy Policy**  

At [Your Company Name], we are committed to protecting the privacy and security of our European customers’ personal data in accordance with the General Data P...


In [27]:

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate


#oneshot
def main():
    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    ) 

    example_policy = """"REFUND POLICY\n1. Eligibility: Within 30 days of purchase\n2. Conditions: Product unused and in original packaging\n3. Process: Submit request via support@company.com\n4. Timeline: Refund processed within 5-7 business days\n5. Exceptions: Digital products and custom orders non-refundable"""  # Replace ___ with: "REFUND POLICY\n1. Eligibility: Within 30 days of purchase\n2. Conditions: Product unused and in original packaging\n3. Process: Submit request via support@company.com\n4. Timeline: Refund processed within 5-7 business days\n5. Exceptions: Digital products and custom orders non-refundable"

    
    print(example_policy)

    
    one_shot_template = PromptTemplate(
        template="""Here's an example of our policy format:

        {example}

        Now write a {policy_type} policy following this EXACT format with numbered sections:""",
        input_variables=["example", "policy_type"] 
    )

    

    formatted_prompt = one_shot_template.format(
        example=example_policy,
        policy_type="remote work"  
    )

    
    response = llm.invoke(formatted_prompt)

    print(response)
    
    has_numbered_sections = any(f"{i}." in response.content for i in range(1, 6))
    has_consistent_structure = all(
        keyword in response.content.lower()
        for keyword in ["eligibility", "conditions", "process", "timeline"]
    )

    if has_numbered_sections and has_consistent_structure:
        print(" follows the exact format of our example")
        
    else:
        print("Format needs adjustment")

    

if __name__ == "__main__":
    main()

"REFUND POLICY
1. Eligibility: Within 30 days of purchase
2. Conditions: Product unused and in original packaging
3. Process: Submit request via support@company.com
4. Timeline: Refund processed within 5-7 business days
5. Exceptions: Digital products and custom orders non-refundable
content='"REMOTE WORK POLICY  \n1. Eligibility: Available to full-time employees in eligible roles  \n2. Conditions: Must maintain productivity and meet performance expectations  \n3. Process: Submit remote work request via HR@company.com for approval  \n4. Timeline: Approval or denial communicated within 3-5 business days  \n5. Exceptions: Roles requiring on-site presence or specific equipment are ineligible"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 98, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_de

In [31]:


import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

def main():
    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    ) 

    
    examples = [
        {
            "input": "refund request",  
            "output": "I understand you'd like a refund. Let me check your order details. Our refund policy allows returns within 30 days. I'll process this for you right away."  # Replace ___ with: "I understand you'd like a refund. Let me check your order details. Our refund policy allows returns within 30 days. I'll process this for you right away."
        },
        {
            "input": "shipping delay",  
            "output": "I apologize for the shipping delay. Let me track your package immediately. I see it's currently in transit and should arrive within 2 days. I'll apply a shipping credit to your account.",
            "input": "password reset",
            "output": "I'll help you reset your password. For security, I've sent a reset link to your registered email. The link expires in 1 hour. Please check your spam folder if you don't see it."  
        }
    ]

    print("Examples loaded:")
    for i, ex in enumerate(examples, 1):
        print(f"  Example {i}: {ex['input']} → {ex['output'][:50]}...")

    
    example_prompt = PromptTemplate(
        template="Customer Issue: {input}\nSupport Response: {output}",
        input_variables=["input", "output"] 
    )

    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="You are a helpful customer support agent. Here are examples of how to respond:",
        suffix="Customer Issue: {input}\nSupport Response:",
        input_variables=["input"]
    )

   

    test_input = "account locked"  

    formatted_prompt = few_shot_prompt.format(input=test_input)

    print(f" New customer issue: {test_input}")
    

  
    response = llm.invoke(formatted_prompt)
    print(f"\n AI Response: {response.content}")

   

    

if __name__ == "__main__":
    main()

Examples loaded:
  Example 1: refund request → I understand you'd like a refund. Let me check you...
  Example 2: password reset → I'll help you reset your password. For security, I...
 New customer issue: account locked

 AI Response: I’m sorry to hear your account is locked. For security reasons, I’ll need to verify some details to unlock it. Could you please confirm the email or phone number associated with your account? Once verified, I can assist you right away.  

Let me know if you’d prefer another verification method, and I’d be happy to help!


In [33]:


import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

def main():
    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )

    
    direct_prompt = "Fix our data retention policy to comply with GDPR."

    print(f"{direct_prompt}")
    direct_response = llm.invoke(direct_prompt)
    print(f"\nDirect response preview: {direct_response.content[:200]}...")


    print("\n Chain-of-Thought Approach")
 
    
    reasoning_steps = """Step 1: Review current GDPR requirements for data retention\nStep 2: Identify gaps in our existing policy\nStep 3: Research industry best practices\nStep 4: Draft specific recommendations\nStep 5: Create implementation timeline"""  # Replace ___ with: "Step 1: Review current GDPR requirements for data retention\nStep 2: Identify gaps in our existing policy\nStep 3: Research industry best practices\nStep 4: Draft specific recommendations\nStep 5: Create implementation timeline"

    print(reasoning_steps)

    cot_template = PromptTemplate(
        template="""To solve this problem, think through it step-by-step:

            {steps}

            Problem: {problem}

            Now, let's work through each step systematically:""",
            input_variables=["steps", "problem"]  
    )

    
    cot_prompt = cot_template.format(
        steps=reasoning_steps,
        problem="Fix our data retention policy to comply with GDPR"  
    )


    cot_response = llm.invoke(cot_prompt)
    print(f"\n Chain-of-Thought Response:\n{cot_response.content[:500]}...")

    
if __name__ == "__main__":
    main()

Fix our data retention policy to comply with GDPR.

Direct response preview: To ensure your **data retention policy** complies with the **General Data Protection Regulation (GDPR)**, follow these key steps:

### **1. Define Legal Basis for Data Retention**
   - **Explicit Cons...

 Chain-of-Thought Approach
Step 1: Review current GDPR requirements for data retention
Step 2: Identify gaps in our existing policy
Step 3: Research industry best practices
Step 4: Draft specific recommendations
Step 5: Create implementation timeline

 Chain-of-Thought Response:
Let’s systematically address each step to fix your data retention policy for GDPR compliance:

---

### **Step 1: Review Current GDPR Requirements for Data Retention**
**Key GDPR Principles to Consider:**
1. **Storage Limitation (Article 5(1)(e))**:  
   - Personal data must be kept no longer than necessary for the purpose it was collected.  
   - Retention periods must be justified and documented.  
2. **Purpose Limitation (Article 5

## Lets put everything together 

In [35]:


import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

def main():
    
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_API_BASE"),
        temperature=0.7,
        max_tokens= 500
    )

    
    test_problem = "Create an employee remote work policy"

    

    results = {}

    zero_shot_result = llm.invoke("test_problem")  

    results["zero_shot"] = zero_shot_result.content
    print(f"Response length: {len(zero_shot_result.content)} characters")
    print(f"Preview: {zero_shot_result.content[:100]}...\n")

    

    one_shot_template = PromptTemplate(
        template="""Example Policy:
VACATION POLICY
1. Eligibility: All full-time employees
2. Accrual: 15 days per year
3. Request: Submit 2 weeks in advance
4. Approval: Manager discretion

Now create: {policy_type}""",
        input_variables=["policy_type"]
    )

    
    one_shot_prompt = one_shot_template.format(
        policy_type="test_problem"  
    )
    one_shot_result = llm.invoke(one_shot_prompt)

    results["one_shot"] = one_shot_result.content
    print(f"Response length: {len(one_shot_result.content)} characters")
    print(f"Preview: {one_shot_result.content[:100]}...\n")

    

    examples = [
        {"policy": "sick leave", "format": "1. Coverage: 10 days/year\n2. Documentation: Doctor's note after 3 days"},
        {"policy": "training", "format": "1. Budget: $2000/employee/year\n2. Approval: Required for external courses"},
    ]

    few_shot_template = """Examples of our policy format:
{examples}

Now create: {policy_type}"""

    
    examples_text = "\n".join([f"- {ex['policy']}: {ex['format']}" for ex in examples])
    few_shot_prompt = few_shot_template.format(
        examples=examples_text,
        policy_type="test_problem" 
    )
    few_shot_result = llm.invoke(few_shot_prompt)

    results["few_shot"] = few_shot_result.content
    print(f"Response length: {len(few_shot_result.content)} characters")
    print(f"Preview: {few_shot_result.content[:100]}...\n")

   
    cot_template = PromptTemplate(
        template="""Think through this step-by-step:
1. Consider who needs remote work
2. Define eligibility criteria
3. Set communication requirements
4. Establish work hours and availability
5. Specify equipment and security needs

Problem: {problem}

Work through each step to create the policy:""",
        input_variables=["problem"]
    )

    cot_prompt = cot_template.format(
        problem="test_problem"  
    )
    cot_result = llm.invoke(cot_prompt)

    results["chain_of_thought"] = cot_result.content
    print(f"Response length: {len(cot_result.content)} characters")
    print(f"Preview: {cot_result.content[:100]}...\n")

 

if __name__ == "__main__":
    main()

Response length: 461 characters
Preview: It seems like you've mentioned "test_problem," but I haven't received enough context to understand w...

Response length: 1461 characters
Preview: Here’s a **test problem** based on the provided **Vacation Policy** example. This problem assesses u...

Response length: 504 characters
Preview: Here’s an example of a policy format for **test_problem**:

- **test_problem**:  
  1. **Identificat...

Response length: 2224 characters
Preview: Here's a step-by-step breakdown to create a remote work policy based on the given problem (`test_pro...

